In [ ]:
# RPB
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

In [ ]:
# Load LaDe dataset (replace path with actual download)
# Dataset available at: https://github.com/wenhaomin/LaDe
try:
    df = pd.read_csv('datasets/delivery_sh.csv')
    print(f"Loaded {len(df)} delivery records from Shanghai")
except FileNotFoundError:
    print("Dataset not found! Using synthetic data instead")
    # Generate synthetic delivery points if dataset unavailable
    np.random.seed(42)
    delivery_points = np.random.uniform(low=[31.0, 121.0], high=[31.3, 121.5], size=(50,2))
    depot = np.array([31.15, 121.3])
else:
    # Preprocess actual data
    delivery_points = df[['lat', 'lng']].values
    depot = np.mean(delivery_points, axis=0)
depot

In [ ]:
df.groupby(["courier_id", "ds"]).agg(count=("order_id", "count")).reset_index().sort_values(by="count").tail()

In [ ]:
delivery_points = df[(df.courier_id==2130) & (df.ds==1010)][['lat', 'lng']].values

In [ ]:
# NUM_POINTS: Defines the genetic makeup and complexity of the problem.
# - Genetic Analogy: The number of genes on a chromosome, where each gene is a delivery point defining the route.
# - Computational Impact: Directly sets the problem's search space; more points exponentially increase complexity.
NUM_POINTS = 121

# POPULATION_SIZE: The size of the evolving gene pool.
# - Genetic Analogy: The number of chromosomes (routes) in each generation, representing the population's genetic diversity.
# - Computational Impact: A larger population improves solution exploration but increases the computational load per generation.
POPULATION_SIZE = 100

# GENERATIONS: The duration of the evolutionary process.
# - Genetic Analogy: The number of evolutionary cycles for the population of routes to adapt and improve their fitness.
# - Computational Impact: More generations allow for better convergence toward an optimal route but increase total runtime.
GENERATIONS = 500

# MUTATION_RATE: The rate of spontaneous genetic change.
# - Genetic Analogy: The probability of a random gene (delivery point) swap, introducing novel traits to escape local optima.
# - Computational Impact: A low-cost operation crucial for maintaining diversity and preventing premature convergence.
MUTATION_RATE = 0.02

# TOURNAMENT_SIZE: The intensity of natural selection.
# - Genetic Analogy: The size of the "survival of the fittest" competition that determines which routes reproduce.
# - Computational Impact: A larger size increases selection pressure, which can speed up convergence but may reduce diversity.
TOURNAMENT_SIZE = 5

# ELITISM_COUNT: The mechanism for preserving elite traits.
# - Genetic Analogy: The number of elite chromosomes (best routes) whose superior genetic code is passed on unchanged.
# - Computational Impact: A computationally cheap way to ensure the best-found solution is never lost, accelerating progress.
ELITISM_COUNT = 2

In [ ]:
# Create distance matrix

# Vertically stack the depot and the selected number of delivery points into a single NumPy array.
# The depot is placed at index 0, and the delivery points follow. This standardized structure
# is essential for easily calculating route distances later. `NUM_POINTS` determines the problem size.
points = np.vstack([depot, delivery_points[:NUM_POINTS]])

# Initialize a square matrix with zeros, with dimensions matching the total number of points.
# This matrix will serve as a lookup table where `distance_matrix[i][j]` will store the
# distance between point `i` and point `j`.
distance_matrix = np.zeros((len(points), len(points)))

# Loop through each point in the `points` array to serve as the starting point.
for i in range(len(points)):
    # Loop through each point again to serve as the destination point.
    for j in range(len(points)):
        # Calculate the straight-line (Euclidean) distance between point `i` and point `j`.
        # The result is stored in the corresponding cell of the matrix. The matrix will be
        # symmetric (distance from i to j is the same as j to i).
        distance_matrix[i][j] = euclidean(points[i], points[j])

In [ ]:
def create_route(points):
    """
    Generates a random delivery route by creating a permutation of all delivery point indices except the depot.
    
    Args:
        points (array-like): List or array of all points, where the depot is at index 0 and delivery points follow.
    
    Returns:
        list: A randomly ordered list of indices representing a delivery route, excluding the depot.
    """
    return random.sample(range(1, len(points)), len(points)-1)

def route_distance(route, distance_matrix):
    """
    Calculates the total travel distance for a delivery route that starts and ends at the depot.
    
    Args:
        route (list): A sequence of point indices representing the order of deliveries, excluding the depot.
    
    Returns:
        float: The total distance of the route, including travel from the depot to the first delivery, 
        between all deliveries, and back to the depot.
    """
    total = distance_matrix[0][route[0]]  # Depot to first point
    for i in range(len(route)-1):
        total += distance_matrix[route[i]][route[i+1]]
    total += distance_matrix[route[-1]][0]  # Last point to depot
    return total

def tournament_selection(population, fitness):
    """
    Selects a parent route from the population using tournament selection.
    
    Args:
        population (list): List of candidate routes (chromosomes) in the current generation.
        fitness (list): List of fitness values corresponding to each route, where lower values indicate better routes.
    
    Returns:
        list: The selected parent route (chromosome) with the lowest fitness among the randomly chosen tournament participants.
    """
    tournament = random.sample(list(zip(population, fitness)), TOURNAMENT_SIZE)
    return min(tournament, key=lambda x: x[1])[0]

def ordered_crossover(parent1, parent2):
    """
    Performs Ordered Crossover (OX) between two parent routes to produce a single offspring.
    
    This genetic operator is designed for permutation-based problems like the Traveling Salesperson Problem. 
    It constructs a child route by first copying a random, contiguous segment from the first parent. 
    The remaining delivery points are then added from the second parent in the order they appear, 
    skipping any points that are already present in the child's route.
    
    Args:
        parent1 (list): The first parent route (chromosome).
        parent2 (list): The second parent route (chromosome).
    
    Returns:
        list: A new child route resulting from the crossover of the two parents.
    """
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = [-1] * len(parent1)
    child[start:end+1] = parent1[start:end+1]
    # Fill remaining positions from parent2
    current_pos = 0
    for gene in parent2:
        if gene not in child:
            while current_pos < len(child) and child[current_pos] != -1:
                current_pos += 1
            if current_pos < len(child):
                child[current_pos] = gene
    return child

def swap_mutation(route):
    """
    Applies swap mutation to a given route based on the MUTATION_RATE.
    
    This operator introduces genetic diversity into the population by randomly selecting two points (genes)
    in a route (chromosome) and swapping their positions. The mutation is probabilistic and only occurs 
    if a random float is less than the globally defined MUTATION_RATE. 
    This helps the algorithm explore new solutions and avoid premature convergence to a local optimum.
    
    Args:
        route (list): The individual route (chromosome) to be subjected to mutation.
    
    Returns:
        list: The route after the potential swap mutation. If the mutation condition is not met, 
        the original route is returned unmodified.
    """
    if random.random() < MUTATION_RATE:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route

In [ ]:
# Initialize population with random routes until population size is reached
population = [create_route(points) for _ in range(POPULATION_SIZE)]
best_fitness = float('inf')  # Initialize best fitness as infinity for comparison
best_route = None            # Placeholder for the best route found
history = []                 # List to store best fitness value of each generation

for gen in range(GENERATIONS):
    # Evaluate fitness of each route in the population
    fitness = [route_distance(route, distance_matrix) for route in population]
    
    # Track the best solution in the current generation
    current_best = min(fitness)  # Find the shortest route distance
    if current_best < best_fitness:
        best_fitness = current_best                           # Update best fitness if current is better
        best_route = population[fitness.index(current_best)]  # Update best route accordingly
    history.append(current_best)  # Record best fitness for this generation
    
    # Prepare to create the next generation of routes
    new_population = []
    
    # Elitism: preserve top elite routes to ensure best solutions are retained
    elite_indices = np.argsort(fitness)[:ELITISM_COUNT]  # Indices of best routes by fitness
    new_population.extend([population[i] for i in elite_indices])  # Add elites to new population

    # Generate the rest of the new population through selection, crossover, and mutation
    while len(new_population) < POPULATION_SIZE:
        parent1 = tournament_selection(population, fitness)  # Select first parent by tournament
        parent2 = tournament_selection(population, fitness)  # Select second parent by tournament
        child = ordered_crossover(parent1, parent2)          # Create child route by crossover
        child = swap_mutation(child)                         # Mutate child route to maintain diversity
        new_population.append(child)                         # Add child to new population

    # Replace old population with the new generation
    population = new_population

In [ ]:
# Visualization
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history)
plt.title('Optimization Progress')
plt.xlabel('Generation')
plt.ylabel('Best Distance (meters)')

plt.subplot(1, 2, 2)
plt.scatter(points[1:,0], points[1:,1], c='blue', label='Delivery Points')
plt.scatter(*depot, c='red', s=100, marker='s', label='Depot')

# Plot best route
route_points = points[best_route]
plt.plot([depot[0], route_points[0][0]], [depot[1], route_points[0][1]], 'k--')
for i in range(len(route_points)-1):
    plt.plot([route_points[i][0], route_points[i+1][0]], 
             [route_points[i][1], route_points[i+1][1]], 'k--')
plt.plot([route_points[-1][0], depot[0]], [route_points[-1][1], depot[1]], 'k--')
plt.legend()
plt.title('Optimized Delivery Route')
plt.tight_layout()
plt.show()

print(f"Best route distance: {best_fitness:.2f} meters")